In [2]:
import json
import struct
import numpy as np
import matplotlib.pyplot as plt
import photonforge as pf
import siepic_forge as siepic
import luxtelligence_lnoi400_forge as lxt
import tidy3d as td

td.config.logging_level = "ERROR"

# Set up technologies
siepic_tech = siepic.ebeam()
basic_tech = pf.basic_technology()
lxt_tech = lxt.lnoi400()
pf.config.default_technology = siepic_tech

# Initialize live viewer for real-time visualization
from photonforge.live_viewer import LiveViewer
viewer = LiveViewer(5004)

# Define simulation parameters
wavelengths = np.linspace(1.53, 1.57, 101)
freqs = pf.C_0 / wavelengths

Starting live viewer at http://localhost:5004


In [3]:
siepic.component_names

{'ANT_MMI_1x2_te1550_3dB_BB',
 'GC_SiN_TE_1310_8degOxide_BB',
 'GC_SiN_TE_1550_8degOxide_BB',
 'GC_TE_1310_8degOxide_BB',
 'GC_TE_1550_8degOxide_BB',
 'GC_TM_1310_8degOxide_BB',
 'GC_TM_1550_8degOxide_BB',
 'crossing_horizontal',
 'crossing_manhattan',
 'ebeam_BondPad',
 'ebeam_DC_2-1_te895',
 'ebeam_DC_te895',
 'ebeam_MMI_2x2_5050_te1310',
 'ebeam_Polarizer_TM_1550_UQAM',
 'ebeam_YBranch_895',
 'ebeam_YBranch_te1310',
 'ebeam_adiabatic_te1550',
 'ebeam_adiabatic_tm1550',
 'ebeam_bdc_te1550',
 'ebeam_crossing4',
 'ebeam_gc_te1550',
 'ebeam_gc_te895',
 'ebeam_gc_tm1550',
 'ebeam_routing_taper_te1550_w=500nm_to_w=3000nm_L=20um',
 'ebeam_routing_taper_te1550_w=500nm_to_w=3000nm_L=40um',
 'ebeam_splitter_swg_assist_te1310',
 'ebeam_splitter_swg_assist_te1550',
 'ebeam_terminator_SiN_1550',
 'ebeam_terminator_SiN_te895',
 'ebeam_terminator_te1310',
 'ebeam_terminator_te1550',
 'ebeam_terminator_tm1550',
 'ebeam_y_1550',
 'ebeam_y_adiabatic',
 'ebeam_y_adiabatic_500pin',
 'taper_SiN_750_3000

In [4]:
siepic.component("taper_si_simm_1310")

In [5]:
@pf.parametric_component
def parametric_ring_resonator(*, port_spec, coupling_distance=0.7, radius=5, bus_length=5, coupling_length=0, euler_fraction=0, model_kwargs={}):
    if isinstance(port_spec, str):
        port_spec = pf.config.default_technology.ports[port_spec]

    ring_coupler = pf.parametric.ring_coupler(
        port_spec=port_spec, 
        coupling_distance=coupling_distance, 
        radius=radius,
        bus_length=bus_length,
        coupling_length=coupling_length,
        euler_fraction=euler_fraction
    )

    ring_resonator = pf.Component("ring_resonator")
    
    bottom_coupler_ref = ring_resonator.add_reference(ring_coupler)
    top_coupler_ref = ring_resonator.add_reference(ring_coupler)
    
    bottom_coupler_ref.connect("P1", top_coupler_ref["P3"])
    bottom_coupler_ref.connect("P3", top_coupler_ref["P1"])

    ring_resonator.add_port(ring_resonator.detect_ports([port_spec]))
    ring_resonator.add_model(pf.Tidy3DModel(**model_kwargs), "Tidy3D")
    ring_resonator.add_model(pf.CircuitModel(), "Circuit")

    return ring_resonator

# viewer(parametric_ring_resonator(port_spec="Rib_TE_1550_500"))

In [6]:
class ThermalModel(pf.Model):
    def __init__(self, n_complex, voltage=0, coefficient=3e-4):
        super().__init__(
            n_complex=n_complex,
            voltage=voltage,
            coefficient=coefficient,
        )
        self.n_complex = np.array(n_complex, ndmin=2)
        self.voltage = voltage
        self.coefficient = coefficient

    def __copy__(self):
        return ThermalModel(self.n_complex, self.voltage, self.coefficient)

    def __deepcopy__(self, memo=None):
        # n_complex is an array, so we want to make sure to create a deep copy of it.
        # Other values (voltage and coefficient) are immutable (floats), so we can use them directly.
        return ThermalModel(self.n_complex.copy(), self.voltage, self.coefficient)

    def __repr__(self):
        return f"ThermalModel({self.n_complex!r}, {self.voltage!r}, {self.coefficient!r})"

    def __str__(self):
        return f"ThermalModel at {self.voltage} V"

    @property
    def as_bytes(self):
        coeffs = struct.pack("<2d", self.voltage, self.coefficient)
        shape = struct.pack("<2l", *self.n_complex.shape)
        n_data = self.n_complex.astype(complex).tobytes()
        # Add version 0 as first byte
        return b"\x00" + coeffs + shape + n_data

    @classmethod
    def from_bytes(cls, byte_repr):
        version = byte_repr[0]
        if version != 0:
            raise RuntimeError(f"Incompatible version for ThermalModel: {version}")

        byte_repr = byte_repr[1:]
        fmt = "<2d2l"
        head_len = struct.calcsize(fmt)
        voltage, coefficient, rows, cols = struct.unpack(fmt, byte_repr[:head_len])

        byte_repr = byte_repr[head_len:]
        n_complex = np.frombuffer(byte_repr, dtype=complex).reshape((rows, cols))

        return cls(n_complex, voltage, coefficient)

    @pf.cache_s_matrix
    def start(self, component, frequencies, voltage=None, **kwargs):
        # Allow overriding voltage as an `s_matrix` kwarg too
        if voltage is None:
            voltage = self.voltage
        n_complex = self.n_complex + self.coefficient * voltage**2
        wg_model = pf.WaveguideModel(n_complex)
        return wg_model.start(component, frequencies, **kwargs)


pf.register_model_class(ThermalModel)

In [7]:
@pf.parametric_component
def create_tunable_resonator(*, name="MRR", port_spec, coupling_distance=0.7, radius=5, ps_length=5):
    if isinstance(port_spec, str):
        port_spec = pf.config.default_technology.ports[port_spec]

    # Bend and Straight waveguide to form the microring
    phase_shifter = pf.parametric.straight(name=f"{name}_ps", port_spec=port_spec, length=ps_length)
    straight = pf.parametric.straight(port_spec=port_spec, length=ps_length)
    coupler = pf.parametric.ring_coupler(port_spec=port_spec, coupling_distance=coupling_distance, radius=radius)

    # Solve for the port mode of the waveguide and extract the complex refractive index
    alpha = 10
    kappa = (alpha * wavelengths * 1e-4 * np.log(10)) / (40 * np.pi)
    mode_solver = pf.port_modes(port=phase_shifter.ports["P0"], frequencies=freqs)
    n_complex = mode_solver.data.n_complex.values.T + 1j * kappa  # add propagation loss

    thermal_model = ThermalModel(n_complex=n_complex)
    phase_shifter.add_model(thermal_model, "Thermal")

    tunable_resonator = pf.Component("tunable_resonator")

    # form main microring structure
    cp1_ref = tunable_resonator.add_reference(coupler)
    cp2_ref = tunable_resonator.add_reference(coupler)
    ps1_ref = tunable_resonator.add_reference(phase_shifter)
    ps2_ref = tunable_resonator.add_reference(phase_shifter)
    # st1_ref = tunable_resonator.add_reference(straight)

    ps1_ref.connect("P0", cp1_ref["P1"])
    # st1_ref.connect("P0", cp1_ref["P3"])
    ps2_ref.connect("P0", cp1_ref["P3"])
    cp2_ref.connect("P3", ps1_ref["P1"])

    # Define the heater and electrical path
    terminal_width = 10

    # ## HORIZONTAL HEATERS
    # heater = (
    #     pf.Path((-radius-1.5, ps_length/2), ps_length)
    #     .segment((radius+1.5, ps_length/2), ps_length)
    # )

    # terminal_vp = (
    #     pf.Path((-radius-1.5 - terminal_width, ps_length/2), terminal_width)
    #     .segment((-radius-1.5, ps_length/2), terminal_width)
    # )

    # terminal_vn = (
    #     pf.Path((radius+1.5, ps_length/2), terminal_width)
    #     .segment((radius+1.5 + terminal_width, ps_length/2), terminal_width)
    # )

    # tunable_resonator.add((11,0), heater)
    # tunable_resonator.add((12,0), terminal_vp)
    # tunable_resonator.add((12,0), terminal_vn)
    # tunable_resonator.add_terminal(pf.Terminal((12,0), terminal_vp), "VP")
    # tunable_resonator.add_terminal(pf.Terminal((12,0), terminal_vn), "VN")

    ## VERTICAL HEATERS
    heater_left = (
        pf.Path((-radius, 0), 3)
        .segment((-radius, ps_length), 3)
    )

    heater_right = (
        pf.Path((radius, 0), 3)
        .segment((radius, ps_length), 3)
    )

    terminal_vp = (
        pf.Path((0, ps_length+radius), 2*radius+terminal_width)
        .segment((0, ps_length+radius+terminal_width), 2*radius+terminal_width)
    )

    routing_top_left = (
        pf.Path((-radius, ps_length+radius), terminal_width)
        .segment((-radius, ps_length), 3)
    )

    routing_bott_left = (
        pf.Path((-radius, -radius), terminal_width)
        .segment((-radius, 0), 3)
    )

    terminal_vn = (
        pf.Path((0, -radius), 2*radius+terminal_width)
        .segment((0, -radius-terminal_width), 2*radius+terminal_width)
    )

    routing_top_right = (
        pf.Path((radius, ps_length+radius), terminal_width)
        .segment((radius, ps_length), 3)
    )

    routing_bott_right = (
        pf.Path((radius, -radius), terminal_width)
        .segment((radius, 0), 3)
    )

    tunable_resonator.add((11,0), heater_left)
    tunable_resonator.add((11,0), heater_right)
    tunable_resonator.add((12,0), terminal_vp)
    tunable_resonator.add((12,0), terminal_vn)
    tunable_resonator.add((12,0), routing_top_left)
    tunable_resonator.add((12,0), routing_bott_left)
    tunable_resonator.add((12,0), routing_top_right)
    tunable_resonator.add((12,0), routing_bott_right)
    tunable_resonator.add_terminal(pf.Terminal((12,0), terminal_vp), "VP")
    tunable_resonator.add_terminal(pf.Terminal((12,0), terminal_vn), "VN")


    ## COMPONENT MODELS
    tunable_resonator.add_port(tunable_resonator.detect_ports([port_spec]))
    tunable_resonator.add_model(pf.CircuitModel(), "CircuitModel")

    port_symmetries = [
        ("P2", "P3", "P0", "P1"),
        ("P1", "P0", "P3", "P2"),
    ]

    field_monitor = td.FieldMonitor(
        center=(0, 0, 0.11), size=(td.inf, td.inf, 0), freqs=[freqs.mean()], name="field"
    )

    tunable_resonator.add_model(pf.Tidy3DModel(port_symmetries=port_symmetries, monitors=[field_monitor]), "Tidy3DModel")

    return tunable_resonator

tunable_resonator = create_tunable_resonator(port_spec="Rib_TE_1550_500", radius=7, coupling_distance=0.6, ps_length=6)
# viewer(tunable_resonator)

Loading cached simulation from C:\Users\ASUS\.tidy3d\pf_cache\VON\ms_info-R27ISU77BQ5WMHN2UYHOYDRCUU24MQCYZA4HFGIWYAQ3DHQAF6FA.json.
Progress: 100%


In [8]:
# Define the width of the regular silicon strip waveguide (µm)
w_strip = 0.5

# Define the width of the subwavelength grating (SWG) equivalent waveguide (µm)
w_swg = 0.555

# Create a port specification for the strip waveguide
port_spec_strip = pf.PortSpec(
    description="Multi mode strip",  # descriptive label for identification
    width=2.5,  # total port width (simulation window)
    num_modes=2,  # number of supported optical modes
    target_neff=3.5,  # target effective index for solver initialization
    limits=(-1.5, 1.22),  # vertical boundaries of the simulation domain
    path_profiles=[
        (w_strip, 0, (1, 0))
    ],  # waveguide path geometry (width, offset, direction)
)

# Create a port specification for the SWG (subwavelength) waveguide
port_spec_swg = pf.PortSpec(
    description="Multi mode swg",
    width=2.5,
    num_modes=2,
    target_neff=3.5,
    limits=(-1.5, 1.22),
    path_profiles=[(w_swg, 0, (1, 0))],
)

In [9]:
# Define a parametric component for the SWG-assisted PBS
@pf.parametric_component
def create_pbs(
    *,
    spec_strip=port_spec_strip,
    spec_swg=port_spec_swg,
    w_corrugation=0.130,
    gap=0.200,
    swg_period=0.240,
    swg_duty_cycle=0.5,
    n_periods=30,
    s_bend_length=6,
    s_bend_offset=2,
):
    # Resolve string-based specs from the default technology if needed
    if isinstance(spec_strip, str):
        spec_strip = pf.config.default_technology.ports[spec_strip]
    if isinstance(spec_swg, str):
        spec_swg = pf.config.default_technology.ports[spec_swg]

    # Extract effective silicon widths from the provided PortSpecs
    w_strip, _ = spec_strip.path_profile_for("Si")
    w_swg, _ = spec_swg.path_profile_for("Si")

    # Compute total coupling length from SWG period and number of periods
    coupling_length = n_periods * swg_period

    # Create one SWG unit cell component consisting of ridge + groove
    unit_cell = pf.Component("SWG Period")

    # Build the ridge and groove rectangles for one period
    ridge_length = swg_duty_cycle * swg_period
    ridge = pf.Rectangle((0, -w_swg / 2), (ridge_length, w_swg / 2))
    groove = pf.Rectangle(
        (ridge_length, -w_swg / 2), (swg_period, w_swg / 2 - w_corrugation)
    )

    # Add ridge and groove into the unit cell along +x
    unit_cell.add((1, 0), ridge, groove)

    # Create the parent PBS component
    pbs = pf.Component("PBS")

    # Instantiate the SWG array by repeating the unit cell N times
    swg_array = pf.Reference(
        component=unit_cell, columns=n_periods, rows=1, spacing=(swg_period, 0.0)
    )

    # Add a short straight continuation for the SWG waveguide after coupling region
    swg_wg = pf.Rectangle(
        (coupling_length, -w_swg / 2),
        (coupling_length + s_bend_length + 0.1, w_swg / 2),
    )

    # Route the strip waveguide: lead-in, straight through coupler, then S-bend
    strip_wg = (
        pf.Path(origin=(-1.0, gap + (w_swg + w_strip) / 2), width=w_strip)
        .segment(endpoint=(coupling_length + 1.0, 0), relative=True)
        .s_bend(endpoint=(s_bend_length, s_bend_offset), relative=True)
    )

    # Add geometry to the PBS component (placed along +x)
    pbs.add((1, 0), swg_array, strip_wg, swg_wg)

    # Detect and add ports for strip (input) and SWG (output on +x boundary)
    pbs.add_port(pbs.detect_ports([spec_strip]))
    pbs.add_port(pbs.detect_ports([spec_swg], on_boundary="+x"))

    # Attach a Tidy3D model for simulation
    pbs.add_model(
        pf.Tidy3DModel(
            monitors=[
                td.FieldMonitor(
                    name="field",
                    center=(0, 0, 0.11),
                    size=(td.inf, td.inf, 0),
                    freqs=[pf.C_0 / 1.55],
                )
            ],
        )
    )
    return pbs


# Instantiate and view the PBS component with default parameters
pbs = create_pbs()
# viewer(pbs)

In [10]:
pf.config.default_technology.ports

Name,Classification,Description,Width (μm),Limits (μm),Radius (μm),Modes,Target n_eff,Path profiles (μm),Voltage path,Current path
MM_SiN_TE_1550_3000,optical,"Multimode SiN Strip TE 1550 nm,…… w=3000 nm",8,"-2.5, 2.9",0,7,2.1,'SiN': 3,,
MM_TE_1550_2000,optical,"Multimode Strip TE 1550 nm,…… w=2000 nm",6,"-2, 2.22",0,12,3.5,'Si': 2,,
MM_TE_1550_3000,optical,"Multimode Strip TE 1550 nm,…… w=3000 nm",6,"-2, 2.22",0,17,3.5,'Si': 3,,
Rib_TE_1310_350,optical,"Rib (90 nm slab) TE 1310 nm,…… w=350 nm",2.35,"-1, 1.22",0,1,3.5,"'Si': 0.35, 'Si…… slab': 3",,
Rib_TE_1550_500,optical,"Rib (90 nm slab) TE 1550 nm,…… w=500 nm",2.5,"-1, 1.22",0,1,3.5,"'Si': 0.5, 'Si…… slab': 3",,
SiN_TE-TM_1550_1000,optical,"SiN Strip TM 1550 nm, w=1000 nm",3,"-1.5, 1.9",0,2,2.1,'SiN': 1,,
SiN_TE_1310_750,optical,"SiN Strip TE 1310 nm, w=750 nm",3,"-1, 1.4",0,1,2.1,'SiN': 0.75,,
SiN_TE_1310_800,optical,"SiN Strip TE 1310 nm, w=800 nm",3,"-1, 1.4",0,1,2.1,'SiN': 0.8,,
SiN_TE_1550_1000,optical,"SiN Strip TE 1550 nm, w=1000 nm",3,"-1, 1.4",0,1,2.1,'SiN': 1,,
SiN_TE_1550_750,optical,"SiN Strip TE 1550 nm, w=750 nm",3,"-1, 1.4",0,1,2.1,'SiN': 0.75,,


In [11]:
@pf.parametric_component
def create_tunable_mzi(*, name="tunable_mzi", port_spec, coupling_distance=0.7, ps_length=5):
    if isinstance(port_spec, str):
        port_spec = pf.config.default_technology.ports[port_spec]

    # Bend and Straight waveguide to form the microring
    phase_shifter = pf.parametric.straight(name=f"{name}_ps", port_spec=port_spec, length=ps_length)
    straight = pf.parametric.straight(port_spec=port_spec, length=ps_length)
    bend = pf.parametric.bend(port_spec=port_spec, radius=5)
    coupler = pf.parametric.dual_ring_coupler(port_spec=port_spec, coupling_distance=coupling_distance, radius=5)

    # Solve for the port mode of the waveguide and extract the complex refractive index
    alpha = 10
    kappa = (alpha * wavelengths * 1e-4 * np.log(10)) / (40 * np.pi)
    mode_solver = pf.port_modes(port=phase_shifter.ports["P0"], frequencies=freqs)
    n_complex = mode_solver.data.n_complex.values.T + 1j * kappa  # add propagation loss

    thermal_model = ThermalModel(n_complex=n_complex)
    phase_shifter.add_model(thermal_model, "Thermal")

    tunable_mzi = pf.Component("tunable_mzi")

    # form main microring structure
    cp1_ref = tunable_mzi.add_reference(coupler)
    cp2_ref = tunable_mzi.add_reference(coupler)
    ps1_ref = tunable_mzi.add_reference(phase_shifter)
    # ps2_ref = tunable_mzi.add_reference(phase_shifter)
    bend1_ref = tunable_mzi.add_reference(bend)
    bend2_ref = tunable_mzi.add_reference(bend) 
    bend3_ref = tunable_mzi.add_reference(bend)
    bend4_ref = tunable_mzi.add_reference(bend) 
    str1_ref = tunable_mzi.add_reference(straight)
    # str2_ref = tunable_mzi.add_reference(straight)

    #Coupler Kiri
    bend1_ref.connect("P1", cp1_ref["P3"])
    bend2_ref.connect("P0", cp1_ref["P2"])
    #Tengah
    ps1_ref.connect("P0", bend1_ref["P0"])
    str1_ref.connect("P0", bend2_ref["P1"])
    bend3_ref.connect("P1", ps1_ref["P1"])
    bend4_ref.connect("P0", str1_ref["P1"])
    #Coupler Kanan
    cp2_ref.connect("P1", bend3_ref["P0"])
    cp2_ref.connect("P0", bend4_ref["P1"])

    # st1_ref = tunable_mzi.add_reference(straight)
    # ps1_ref.connect("P0", cp1_ref["P1"])
    # # st1_ref.connect("P0", cp1_ref["P3"])
    # ps2_ref.connect("P0", cp1_ref["P3"])
    # cp2_ref.connect("P3", ps1_ref["P1"])

    # Define the heater and electrical path
    # terminal_width = 10

    # ## HORIZONTAL HEATERS
    # heater = (
    #     pf.Path((-radius-1.5, ps_length/2), ps_length)
    #     .segment((radius+1.5, ps_length/2), ps_length)
    # )

    # terminal_vp = (
    #     pf.Path((-radius-1.5 - terminal_width, ps_length/2), terminal_width)
    #     .segment((-radius-1.5, ps_length/2), terminal_width)
    # )

    # terminal_vn = (
    #     pf.Path((radius+1.5, ps_length/2), terminal_width)
    #     .segment((radius+1.5 + terminal_width, ps_length/2), terminal_width)
    # )

    # tunable_mzi.add((11,0), heater)
    # tunable_mzi.add((12,0), terminal_vp)
    # tunable_mzi.add((12,0), terminal_vn)
    # tunable_mzi.add_terminal(pf.Terminal((12,0), terminal_vp), "VP")
    # tunable_mzi.add_terminal(pf.Terminal((12,0), terminal_vn), "VN")

    ## VERTICAL HEATERS
    # heater_left = (
    #     pf.Path((-radius, 0), 3)
    #     .segment((-radius, ps_length), 3)
    # )

    # heater_right = (
    #     pf.Path((radius, 0), 3)
    #     .segment((radius, ps_length), 3)
    # )

    # terminal_vp = (
    #     pf.Path((0, ps_length+radius), 2*radius+terminal_width)
    #     .segment((0, ps_length+radius+terminal_width), 2*radius+terminal_width)
    # )

    # routing_top_left = (
    #     pf.Path((-radius, ps_length+radius), terminal_width)
    #     .segment((-radius, ps_length), 3)
    # )

    # routing_bott_left = (
    #     pf.Path((-radius, -radius), terminal_width)
    #     .segment((-radius, 0), 3)
    # )

    # terminal_vn = (
    #     pf.Path((0, -radius), 2*radius+terminal_width)
    #     .segment((0, -radius-terminal_width), 2*radius+terminal_width)
    # )

    # routing_top_right = (
    #     pf.Path((radius, ps_length+radius), terminal_width)
    #     .segment((radius, ps_length), 3)
    # )

    # routing_bott_right = (
    #     pf.Path((radius, -radius), terminal_width)
    #     .segment((radius, 0), 3)
    # )

    # tunable_mzi.add((11,0), heater_left)
    # tunable_mzi.add((11,0), heater_right)
    # tunable_mzi.add((12,0), terminal_vp)
    # tunable_mzi.add((12,0), terminal_vn)
    # tunable_mzi.add((12,0), routing_top_left)
    # tunable_mzi.add((12,0), routing_bott_left)
    # tunable_mzi.add((12,0), routing_top_right)
    # tunable_mzi.add((12,0), routing_bott_right)
    # tunable_mzi.add_terminal(pf.Terminal((12,0), terminal_vp), "VP")
    # tunable_mzi.add_terminal(pf.Terminal((12,0), terminal_vn), "VN")


    ## COMPONENT MODELS
    tunable_mzi.add_port(tunable_mzi.detect_ports([port_spec]))
    tunable_mzi.add_model(pf.CircuitModel(), "CircuitModel")

    # port_symmetries = [
    #     ("P2", "P3", "P0", "P1"),
    #     ("P1", "P0", "P3", "P2"),
    # ]

    # field_monitor = td.FieldMonitor(
    #     center=(0, 0, 0.11), size=(td.inf, td.inf, 0), freqs=[freqs.mean()], name="field"
    # )

    # tunable_mzi.add_model(pf.Tidy3DModel(port_symmetries=port_symmetries, monitors=[field_monitor]), "Tidy3DModel")

    return tunable_mzi

tunable_mzi = create_tunable_mzi(port_spec="Rib_TE_1550_500", coupling_distance=0.6, ps_length=6)
# viewer(tunable_mzi)

Progress: 100%


In [12]:
@pf.parametric_component
def create_tunable_mzi_siepic(*, name="tunable_mzi", coupling_distance=0.7, coupling_length=4.8, ps_length=5):

    # Bend and Straight waveguide to form the microring
    phase_shifter = pf.parametric.straight(name=f"{name}_ps", port_spec="Rib_TE_1550_500", length=ps_length)
    straight = pf.parametric.straight(port_spec="TE_1550_500", length=ps_length+10)
    bend = pf.parametric.bend(port_spec="TE_1550_500", radius=5)
    trans = pf.parametric.transition(port_spec1="TE_1550_500", port_spec2="Rib_TE_1550_500", length=5)
    coupler = pf.parametric.dual_ring_coupler(
        port_spec="TE_1550_500", coupling_distance=coupling_distance, coupling_length=coupling_length, radius=5, 
        tidy3d_model_kwargs={
            # These symmetries are not correct for arbitrary ports.
            # They can be used in this case for the fundamental mode.
            "port_symmetries": [
                ("P1", "P0", "P3", "P2"),
                ("P2", "P3", "P0", "P1"),
                ("P3", "P2", "P1", "P0"),
            ],
        },
    )

    # Solve for the port mode of the waveguide and extract the complex refractive index
    alpha = 10
    kappa = (alpha * wavelengths * 1e-4 * np.log(10)) / (40 * np.pi)
    mode_solver = pf.port_modes(port=phase_shifter.ports["P0"], frequencies=freqs)
    n_complex = mode_solver.data.n_complex.values.T + 1j * kappa  # add propagation loss

    thermal_model = ThermalModel(n_complex=n_complex)
    phase_shifter.add_model(thermal_model, "Thermal")

    tunable_mzi = pf.Component("tunable_mzi")

    # form main microring structure
    cp1_ref = tunable_mzi.add_reference(coupler)
    cp2_ref = tunable_mzi.add_reference(coupler)
    ps1_ref = tunable_mzi.add_reference(phase_shifter)
    bend1_ref = tunable_mzi.add_reference(bend)
    bend2_ref = tunable_mzi.add_reference(bend) 
    bend3_ref = tunable_mzi.add_reference(bend)
    bend4_ref = tunable_mzi.add_reference(bend) 
    str1_ref = tunable_mzi.add_reference(straight)
    trans1_ref = tunable_mzi.add_reference(trans)
    trans2_ref = tunable_mzi.add_reference(trans)

    #Coupler Kiri
    bend1_ref.connect("P1", cp1_ref["P3"])
    bend2_ref.connect("P0", cp1_ref["P2"])
    #Tengah
    trans1_ref.connect("P0", bend1_ref["P0"])
    ps1_ref.connect("P0", trans1_ref["P1"])
    str1_ref.connect("P0", bend2_ref["P1"])

    trans2_ref.connect("P1", ps1_ref["P1"])
    bend3_ref.connect("P1", trans2_ref["P0"])
    bend4_ref.connect("P0", str1_ref["P1"])
    #Coupler Kanan
    cp2_ref.connect("P1", bend3_ref["P0"])
    cp2_ref.connect("P0", bend4_ref["P1"])

    # st1_ref = tunable_mzi.add_reference(straight)
    # ps1_ref.connect("P0", cp1_ref["P1"])
    # # st1_ref.connect("P0", cp1_ref["P3"])
    # ps2_ref.connect("P0", cp1_ref["P3"])
    # cp2_ref.connect("P3", ps1_ref["P1"])

    # Define the heater and electrical path
    # terminal_width = 10

    # ## HORIZONTAL HEATERS
    # heater = (
    #     pf.Path((-radius-1.5, ps_length/2), ps_length)
    #     .segment((radius+1.5, ps_length/2), ps_length)
    # )

    # terminal_vp = (
    #     pf.Path((-radius-1.5 - terminal_width, ps_length/2), terminal_width)
    #     .segment((-radius-1.5, ps_length/2), terminal_width)
    # )

    # terminal_vn = (
    #     pf.Path((radius+1.5, ps_length/2), terminal_width)
    #     .segment((radius+1.5 + terminal_width, ps_length/2), terminal_width)
    # )

    # tunable_mzi.add((11,0), heater)
    # tunable_mzi.add((12,0), terminal_vp)
    # tunable_mzi.add((12,0), terminal_vn)
    # tunable_mzi.add_terminal(pf.Terminal((12,0), terminal_vp), "VP")
    # tunable_mzi.add_terminal(pf.Terminal((12,0), terminal_vn), "VN")

    ## VERTICAL HEATERS
    # heater_left = (
    #     pf.Path((-radius, 0), 3)
    #     .segment((-radius, ps_length), 3)
    # )

    # heater_right = (
    #     pf.Path((radius, 0), 3)
    #     .segment((radius, ps_length), 3)
    # )

    # terminal_vp = (
    #     pf.Path((0, ps_length+radius), 2*radius+terminal_width)
    #     .segment((0, ps_length+radius+terminal_width), 2*radius+terminal_width)
    # )

    # routing_top_left = (
    #     pf.Path((-radius, ps_length+radius), terminal_width)
    #     .segment((-radius, ps_length), 3)
    # )

    # routing_bott_left = (
    #     pf.Path((-radius, -radius), terminal_width)
    #     .segment((-radius, 0), 3)
    # )

    # terminal_vn = (
    #     pf.Path((0, -radius), 2*radius+terminal_width)
    #     .segment((0, -radius-terminal_width), 2*radius+terminal_width)
    # )

    # routing_top_right = (
    #     pf.Path((radius, ps_length+radius), terminal_width)
    #     .segment((radius, ps_length), 3)
    # )

    # routing_bott_right = (
    #     pf.Path((radius, -radius), terminal_width)
    #     .segment((radius, 0), 3)
    # )

    # tunable_mzi.add((11,0), heater_left)
    # tunable_mzi.add((11,0), heater_right)
    # tunable_mzi.add((12,0), terminal_vp)
    # tunable_mzi.add((12,0), terminal_vn)
    # tunable_mzi.add((12,0), routing_top_left)
    # tunable_mzi.add((12,0), routing_bott_left)
    # tunable_mzi.add((12,0), routing_top_right)
    # tunable_mzi.add((12,0), routing_bott_right)
    # tunable_mzi.add_terminal(pf.Terminal((12,0), terminal_vp), "VP")
    # tunable_mzi.add_terminal(pf.Terminal((12,0), terminal_vn), "VN")


    ## COMPONENT MODELS
    tunable_mzi.add_port(tunable_mzi.detect_ports(["TE_1550_500"]))
    tunable_mzi.add_model(pf.CircuitModel(), "CircuitModel")

    port_symmetries = [
        ("P1", "P0", "P3", "P2"),
        ("P2", "P3", "P0", "P1"),
        ("P3", "P2", "P1", "P0"),
    ]

    field_monitor = td.FieldMonitor(
        center=(0, 0, 0.11), size=(td.inf, td.inf, 0), freqs=[freqs.mean()], name="field"
    )

    tunable_mzi.add_model(pf.Tidy3DModel(port_symmetries=port_symmetries, monitors=[field_monitor]), "Tidy3DModel")

    return tunable_mzi

tunable_mzi = create_tunable_mzi_siepic(ps_length=100)
# viewer(tunable_mzi)

Progress: 100%


In [13]:
@pf.parametric_component
def create_tunable_mzi_siepic(*, name, coupling_distance=0.6, coupling_length=5.35, ps_length=5):

    # Bend and Straight waveguide to form the microring
    phase_shifter = pf.parametric.straight(name=f"ps", port_spec="Rib_TE_1550_500", length=ps_length)
    straight = pf.parametric.straight(port_spec="TE_1550_500", length=ps_length+10)
    bend = pf.parametric.bend(port_spec="TE_1550_500", radius=5)
    trans = pf.parametric.transition(port_spec1="TE_1550_500", port_spec2="Rib_TE_1550_500", length=5)
    coupler = pf.parametric.dual_ring_coupler(
        port_spec="TE_1550_500", coupling_distance=coupling_distance, coupling_length=coupling_length, radius=5, 
        tidy3d_model_kwargs={
            # These symmetries are not correct for arbitrary ports.
            # They can be used in this case for the fundamental mode.
            "port_symmetries": [
                ("P1", "P0", "P3", "P2"),
                ("P2", "P3", "P0", "P1"),
                ("P3", "P2", "P1", "P0"),
            ],
        },
    )

    # Solve for the port mode of the waveguide and extract the complex refractive index
    alpha = 10
    kappa = (alpha * wavelengths * 1e-4 * np.log(10)) / (40 * np.pi)
    mode_solver = pf.port_modes(port=phase_shifter.ports["P0"], frequencies=freqs)
    n_complex = mode_solver.data.n_complex.values.T + 1j * kappa  # add propagation loss

    thermal_model = ThermalModel(n_complex=n_complex)
    phase_shifter.add_model(thermal_model, "Thermal")

    tunable_mzi = pf.Component(name)

    # form main microring structure
    cp1_ref = tunable_mzi.add_reference(coupler)
    cp2_ref = tunable_mzi.add_reference(coupler)
    ps1_ref = tunable_mzi.add_reference(phase_shifter)
    bend1_ref = tunable_mzi.add_reference(bend)
    bend2_ref = tunable_mzi.add_reference(bend) 
    bend3_ref = tunable_mzi.add_reference(bend)
    bend4_ref = tunable_mzi.add_reference(bend) 
    str1_ref = tunable_mzi.add_reference(straight)
    trans1_ref = tunable_mzi.add_reference(trans)
    trans2_ref = tunable_mzi.add_reference(trans)

    #Coupler Kiri
    bend1_ref.connect("P1", cp1_ref["P3"])
    bend2_ref.connect("P0", cp1_ref["P2"])
    #Tengah
    trans1_ref.connect("P0", bend1_ref["P0"])
    ps1_ref.connect("P0", trans1_ref["P1"])
    str1_ref.connect("P0", bend2_ref["P1"])

    trans2_ref.connect("P1", ps1_ref["P1"])
    bend3_ref.connect("P1", trans2_ref["P0"])
    bend4_ref.connect("P0", str1_ref["P1"])
    #Coupler Kanan
    cp2_ref.connect("P1", bend3_ref["P0"])
    cp2_ref.connect("P0", bend4_ref["P1"])


    # Define the heater and electrical path
    terminal_width = 10
    heater_width = 1

    heater = (
        pf.Path((ps1_ref.x_min, ps1_ref.y_mid), heater_width)
        .segment((ps1_ref.x_max, ps1_ref.y_mid), heater_width)
    )

    route_vp = (
        pf.Path((cp1_ref.x_mid-terminal_width/2, ps1_ref.y_mid), terminal_width)
        .segment((cp1_ref.x_mid+terminal_width/2, ps1_ref.y_mid), terminal_width)
        .segment((ps1_ref.x_min, ps1_ref.y_mid), heater_width)
    )

    route_vn = (
        pf.Path((ps1_ref.x_max, ps1_ref.y_mid), heater_width)
        .segment((cp2_ref.x_mid-terminal_width/2, ps1_ref.y_mid), terminal_width)
        .segment((cp2_ref.x_mid+terminal_width/2, ps1_ref.y_mid), terminal_width)
    )

    tunable_mzi.add((11,0), heater)
    tunable_mzi.add((12,0), route_vp)
    tunable_mzi.add((12,0), route_vn)
    tunable_mzi.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), center=(cp1_ref.x_mid, ps1_ref.y_mid))), "VP")
    tunable_mzi.add_terminal(pf.Terminal((12,0), pf.Rectangle(size=(terminal_width, terminal_width), center=(cp2_ref.x_mid, ps1_ref.y_mid))), "VN")

    ## COMPONENT MODELS
    tunable_mzi.add_port(tunable_mzi.detect_ports(["TE_1550_500"]))
    tunable_mzi.add_model(pf.CircuitModel(), "CircuitModel")

    port_symmetries = [
        ("P1", "P0", "P3", "P2"),
        ("P2", "P3", "P0", "P1"),
        ("P3", "P2", "P1", "P0"),
    ]

    field_monitor = td.FieldMonitor(
        center=(0, 0, 0.11), size=(td.inf, td.inf, 0), freqs=[freqs.mean()], name="field"
    )

    tunable_mzi.add_model(pf.Tidy3DModel(port_symmetries=port_symmetries, monitors=[field_monitor]), "Tidy3DModel")

    return tunable_mzi

# tunable_mzi = create_tunable_mzi_siepic(name="mzm", ps_length=99.3)
# viewer(tunable_mzi)

c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\photonforge\parametric_utils.py:206: RuntimeWarning: Component function '__main__.create_tunable_mzi_siepic' previously registered will be overwritten.
  return _decorator(decorated_function)


In [14]:
bob_top = pf.Component("bob_top")
# ring_resonator = parametric_ring_resonator(port_spec="Rib_TE_1550_500", radius=10, bus_length=15, coupling_distance=0.6)
# tunable_resonator = create_tunable_resonator(port_spec="Rib_TE_1550_500", radius=7, coupling_distance=0.6, ps_length=6)
straight = pf.parametric.straight(port_spec="TE_1550_500", length=26)
half_straight = pf.parametric.straight(port_spec="TE_1550_500", length=13)
quarter_straight = pf.parametric.straight(port_spec="TE_1550_500", length=5)
y_branch = siepic.component("ebeam_y_1550")
grating_coupler = siepic.component("ebeam_gc_te1550")
bend = pf.parametric.bend(port_spec="TE_1550_500", radius=5, euler_fraction=0.5, angle=90)
sbend_top = pf.parametric.s_bend(port_spec="TE_1550_500", length=30, offset=-10.85, euler_fraction=0.5)
sbend_bottom = pf.parametric.s_bend(port_spec="TE_1550_500", length=30, offset=10.85, euler_fraction=0.5)
wider_sbend_top = pf.parametric.s_bend(port_spec="TE_1550_500", length=45, offset=-24.45, euler_fraction=0.5)
wider_sbend_bottom = pf.parametric.s_bend(port_spec="TE_1550_500", length=45, offset=24.45, euler_fraction=0.5)
trans = pf.parametric.transition(port_spec1="TE_1550_500", port_spec2="Rib_TE_1550_500", length=5, constant_length=0.5)
trans_pbs = pf.parametric.transition(port_spec1="TE_1550_500", port_spec2="TE_1550_500", length=5, constant_length=0.5)
pbs = create_pbs()
# tunable_mzi = create_tunable_mzi(port_spec="Rib_TE_1550_500", coupling_distance=0.6, ps_length=6)
mzm_base = create_tunable_mzi_siepic(name="mzm_base", ps_length=99.3)
mzm_bit = create_tunable_mzi_siepic(name="mzm_bit", ps_length=99.3)

gc1_ref = bob_top.add_reference(grating_coupler)
# gc2_ref = bob_top.add_reference(grating_coupler)
# gc3_ref = bob_top.add_reference(grating_coupler)

# rr1_ref = bob_top.add_reference(ring_resonator)
# rr2_ref = bob_top.add_reference(ring_resonator)
# rr3_ref = bob_top.add_reference(ring_resonator)

MZI1_ref = bob_top.add_reference(mzm_base)
MZI2_ref = bob_top.add_reference(mzm_bit)
MZI3_ref = bob_top.add_reference(mzm_bit)

# st1_ref = bob_top.add_reference(straight)
# st2_ref = bob_top.add_reference(straight)
# st3_ref = bob_top.add_reference(straight)
# st4_ref = bob_top.add_reference(straight)

hst1_ref = bob_top.add_reference(half_straight)
# hst2_ref = bob_top.add_reference(half_straight)
# hst3_ref = bob_top.add_reference(half_straight)
# hst4_ref = bob_top.add_reference(half_straight)
# hst5_ref = bob_top.add_reference(half_straight)
# hst6_ref = bob_top.add_reference(half_straight)
# hst7_ref = bob_top.add_reference(half_straight)
# hst8_ref = bob_top.add_reference(half_straight)

qst1_ref = bob_top.add_reference(quarter_straight)
qst2_ref = bob_top.add_reference(quarter_straight)

bend1_ref = bob_top.add_reference(bend)
bend2_ref = bob_top.add_reference(bend)
bend3_ref = bob_top.add_reference(bend)

sbt1_ref = bob_top.add_reference(sbend_top)
# # sbt2_ref = bob_top.add_reference(sbend_top)
# # sbt3_ref = bob_top.add_reference(sbend_top)

# sbb1_ref = bob_top.add_reference(sbend_bottom)
# # sbb2_ref = bob_top.add_reference(sbend_bottom)
# # sbb3_ref = bob_top.add_reference(sbend_bottom)

# wsbt1_ref = bob_top.add_reference(wider_sbend_top)
# # wsbt2_ref = bob_top.add_reference(wider_sbend_top)
# # wsbt3_ref = bob_top.add_reference(wider_sbend_top)

# wsbb1_ref = bob_top.add_reference(wider_sbend_bottom)
# wsbb2_ref = bob_top.add_reference(wider_sbend_bottom)
# wsbb3_ref = bob_top.add_reference(wider_sbend_bottom)

y1_ref = bob_top.add_reference(y_branch)
# y2_ref = bob_top.add_reference(y_branch)
# y3_ref = bob_top.add_reference(y_branch)

# trans1_ref = bob_top.add_reference(trans)
# trans2_ref = bob_top.add_reference(trans)
# trans3_ref = bob_top.add_reference(trans)

pbs_ref = bob_top.add_reference(pbs)

#Grating Coupler, Resonator 
hst1_ref.connect("P0",gc1_ref["P0"])
bend1_ref.connect("P1",hst1_ref["P1"])
# trans1_ref.connect("P0",bend1_ref["P0"])
MZI1_ref.connect("P1",bend1_ref["P0"])
#Jalur Atas
# trans2_ref.connect("P1",MZI1_ref["P3"])
bend2_ref.connect("P1",MZI1_ref["P3"])
qst1_ref.connect("P0",bend2_ref["P0"])
#Jalur Bawah, Y Branch
# trans3_ref.connect("P1",MZI1_ref["P2"])
bend3_ref.connect("P0",MZI1_ref["P2"])
qst2_ref.connect("P0",bend3_ref["P1"])

#Rotate, Y Branch
y1_ref.rotate(180)

y1_ref.x_min = qst2_ref.x_max + 10
y1_ref.y_mid = (qst2_ref.y_mid + qst1_ref.y_mid) / 2

route_y1_1 = pf.parametric.route(port1=(qst1_ref, "P1"), port2=(y1_ref, "P1"), radius=5)
route_y1_2 = pf.parametric.route(port1=(qst2_ref, "P1"), port2=(y1_ref, "P2"), radius=5)
bob_top.add_reference(route_y1_1)
bob_top.add_reference(route_y1_2)

#Beam Splitter, Grating Coupler
pbs_ref.connect("P0",y1_ref["P0"])

#Grating Coupler Output 1
gc2_ref.rotate(180)

gc2_ref.x_min = pbs_ref.x_max + 10
gc2_ref.y_mid = pbs_ref.y_max + 15

route_gc2_1 = pf.parametric.route(port1=(pbs_ref, "P1"), port2=(gc2_ref, "P0"), radius=5)
bob_top.add_reference(route_gc2_1)

# gc3_ref.rotate(180)

# gc3_ref.x_min = pbs_ref.x_max + 10
# gc3_ref.y_mid = pbs_ref.y_max + 15

# route_gc3_1 = pf.parametric.route(port1=(pbs_ref, "P2"), port2=(gc3_ref, "P0"), radius=5)
# bob_top.add_reference(route_gc3_1)

#Grating Coupler Output 2
sbt1_ref.connect("P0",pbs_ref["P2"])
gc3_ref.connect("P0",sbt1_ref["P1"])

viewer(bob_top)

Progress: 100%
Progress: 100%


NameError: name 'gc2_ref' is not defined